In [ ]:
# ============================================
# 05_risk_manager.ipynb - Semáforo de riesgo (CON ALPHA vs S&P 500)
# ============================================

import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "data": f"{BASE}/data",
    "clean": f"{BASE}/data/clean"
}

# --- Leer cartera enriquecida ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
total = pf["importe_actual_eur"].sum()

# --- Leer drawdown y retorno de la cartera reconstruida ---
DAILY_PATH = f"{DIRS['reports']}/portfolio_daily_value.csv"
annual_return = 0.0
current_dd = 0.0
alpha = np.nan

if os.path.exists(DAILY_PATH):
    portfolio_daily = pd.read_csv(DAILY_PATH, index_col=0, parse_dates=True)
    current_dd = portfolio_daily["drawdown"].min()

    # Calcular retorno anualizado de la cartera
    first_val = portfolio_daily["valor_mejorado"].iloc[0]
    last_val = portfolio_daily["valor_mejorado"].iloc[-1]
    days = (portfolio_daily.index[-1] - portfolio_daily.index[0]).days
    if days > 0:
        annual_return = (last_val / first_val) ** (252 / days) - 1
else:
    print("⚠️ Archivo portfolio_daily_value.csv no encontrado. Usando valores por defecto.")
    current_dd = 0.0
    annual_return = 0.0

# --- Comparar con S&P 500 ---
BENCHMARK_PATH = f"{DIRS['clean']}/indices_prices_clean.parquet"
sp500_return = np.nan

if os.path.exists(BENCHMARK_PATH):
    try:
        indices = pd.read_parquet(BENCHMARK_PATH)
        if "^GSPC" in indices.columns:
            sp500 = indices["^GSPC"].dropna()
            # Alinear fechas con tu cartera
            if os.path.exists(DAILY_PATH):
                common_dates = portfolio_daily.index.intersection(sp500.index)
                if len(common_dates) > 100:
                    sp500_aligned = sp500[common_dates]
                    sp500_return = (sp500_aligned.iloc[-1] / sp500_aligned.iloc[0]) ** (252 / len(sp500_aligned)) - 1
                    alpha = annual_return - sp500_return
    except Exception as e:
        print(f"⚠️ Error al leer el benchmark: {e}")

# --- Configuración de límites ---
MAX_REGION = 0.30
MAX_ASSET = 0.05
MAX_DD = 0.10
AMBER_DD = 0.09

# --- Límites por activo ---
pf["breach_asset"] = pf["peso_%"] / 100 > MAX_ASSET
breaches_asset = pf[pf["breach_asset"]]

# --- Límites por región ---
exp_region = pf.groupby("region")["importe_actual_eur"].sum().to_frame()
exp_region["peso_%"] = exp_region["importe_actual_eur"] / total * 100
exp_region["breach_region"] = exp_region["peso_%"] / 100 > MAX_REGION
breaches_region = exp_region[exp_region["breach_region"]]

# --- USD no hedged ---
usd_unhedged = pf[(pf["divisa_base"] == "USD") & (pf["hedged"] != "Sí")]["importe_actual_eur"].sum()
usd_unhedged_pct = usd_unhedged / total

# --- Alertas de drawdown ---
amber_alert = current_dd <= -AMBER_DD
red_alert = current_dd <= -MAX_DD

# --- Dashboard ---
risk_dashboard = pd.DataFrame({
    "métrica": [
        "Drawdown actual",
        "Retorno anualizado cartera",
        "Retorno anualizado S&P 500",
        "Alpha vs S&P 500",
        "Alpha objetivo (+5%)",
        "Drawdown ≥ 9% (ámbar)",
        "Drawdown ≥ 10% (rojo)",
        "Activos > 5%",
        "Regiones > 30%",
        "USD no hedged (%)"
    ],
    "valor": [
        f"{current_dd:.1%}",
        f"{annual_return:.1%}",
        f"{sp500_return:.1%}" if pd.notna(sp500_return) else "N/A",
        f"{alpha:.1%}" if pd.notna(alpha) else "N/A",
        "✅" if pd.notna(alpha) and alpha >= 0.05 else "❌",
        amber_alert,
        red_alert,
        len(breaches_asset),
        len(breaches_region),
        f"{usd_unhedged_pct:.1%}"
    ],
    "estado": [
        "✅" if not amber_alert else "⚠️" if not red_alert else "🔴",
        "✅" if annual_return > 0 else "⚠️",
        "N/A",
        "✅" if pd.notna(alpha) and alpha > 0 else "⚠️",
        "✅" if pd.notna(alpha) and alpha >= 0.05 else "⚠️",
        "✅" if not amber_alert else "⚠️",
        "✅" if not red_alert else "🔴",
        "✅" if len(breaches_asset) == 0 else "⚠️",
        "✅" if len(breaches_region) == 0 else "⚠️",
        "✅" if usd_unhedged_pct < 0.10 else "⚠️"
    ]
})

print("=== 🛡️ DASHBOARD DE RIESGO (con Alpha vs S&P 500) ===")
display(risk_dashboard)

# --- Guardar ---
risk_dashboard.to_csv(f"{DIRS['reports']}/risk_dashboard.csv", index=False)
breaches_asset.to_csv(f"{DIRS['reports']}/breaches_asset.csv", index=False)
breaches_region.to_csv(f"{DIRS['reports']}/breaches_region.csv", index=False)

print("\n✅ Risk Manager completado. Archivos guardados en /reports/")

In [1]:
# ============================================
# 05_risk_manager.ipynb - Semáforo de riesgo (MEJORADO + CONTRATO)
# ============================================

import os
import pandas as pd
import numpy as np
import json
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "data": f"{BASE}/data",
    "clean": f"{BASE}/data/clean"
}

# --- Leer cartera enriquecida ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("❌ Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
total = pf["importe_actual_eur"].sum()

if total == 0:
    raise ValueError("❌ El valor total de la cartera es 0.")

# --- Leer drawdown y retorno de la cartera reconstruida ---
DAILY_PATH = f"{DIRS['reports']}/portfolio_daily_value.csv"
annual_return = 0.0
current_dd = 0.0
alpha = np.nan

if os.path.exists(DAILY_PATH):
    try:
        portfolio_daily = pd.read_csv(DAILY_PATH, index_col=0, parse_dates=True)
        if "drawdown" not in portfolio_daily.columns or "valor_mejorado" not in portfolio_daily.columns:
            raise ValueError("Columnas faltantes en portfolio_daily_value.csv")

        current_dd = portfolio_daily["drawdown"].min()

        # Calcular retorno anualizado de la cartera
        first_val = portfolio_daily["valor_mejorado"].iloc[0]
        last_val = portfolio_daily["valor_mejorado"].iloc[-1]
        days = (portfolio_daily.index[-1] - portfolio_daily.index[0]).days
        if days > 0:
            annual_return = (last_val / first_val) ** (252 / days) - 1
    except Exception as e:
        print(f"⚠️ Error al leer portfolio_daily_value.csv: {e}. Usando valores por defecto.")
        current_dd = 0.0
        annual_return = 0.0
else:
    print("⚠️ Archivo portfolio_daily_value.csv no encontrado. Usando valores por defecto.")
    current_dd = 0.0
    annual_return = 0.0

# --- Comparar con S&P 500 ---
BENCHMARK_PATH = f"{DIRS['clean']}/indices_prices_clean.parquet"
sp500_return = np.nan

if os.path.exists(BENCHMARK_PATH):
    try:
        indices = pd.read_parquet(BENCHMARK_PATH)
        if "^GSPC" not in indices.columns:
            print("⚠️ ^GSPC no encontrado en indices_prices_clean.parquet")
        else:
            sp500 = indices["^GSPC"].dropna()
            if os.path.exists(DAILY_PATH):
                try:
                    portfolio_daily = pd.read_csv(DAILY_PATH, index_col=0, parse_dates=True)
                    common_dates = portfolio_daily.index.intersection(sp500.index)
                    if len(common_dates) > 100:
                        sp500_aligned = sp500[common_dates]
                        portfolio_aligned = portfolio_daily.loc[common_dates]["valor_mejorado"]
                        # Calcular retornos anualizados en el mismo período
                        sp500_ret = (sp500_aligned.iloc[-1] / sp500_aligned.iloc[0]) ** (252 / len(sp500_aligned)) - 1
                        portfolio_ret = (portfolio_aligned.iloc[-1] / portfolio_aligned.iloc[0]) ** (252 / len(portfolio_aligned)) - 1
                        sp500_return = sp500_ret
                        alpha = portfolio_ret - sp500_ret
                    else:
                        print(f"⚠️ Pocas fechas comunes ({len(common_dates)}) para calcular alpha")
                except Exception as e:
                    print(f"⚠️ Error al alinear fechas: {e}")
    except Exception as e:
        print(f"⚠️ Error al leer el benchmark: {e}")

# --- Configuración de límites ---
MAX_REGION = 0.30
MAX_ASSET = 0.05
MAX_DD = 0.10
AMBER_DD = 0.09

# --- Límites por activo ---
pf["breach_asset"] = pf["peso_%"] / 100 > MAX_ASSET
breaches_asset = pf[pf["breach_asset"]]

# --- Límites por región ---
exp_region = pf.groupby("region")["importe_actual_eur"].sum().to_frame()
exp_region["peso_%"] = exp_region["importe_actual_eur"] / total * 100
exp_region["breach_region"] = exp_region["peso_%"] / 100 > MAX_REGION
breaches_region = exp_region[exp_region["breach_region"]]

# --- USD no hedged ---
usd_unhedged = pf[(pf["divisa_base"] == "USD") & (pf["hedged"] != "Sí")]["importe_actual_eur"].sum()
usd_unhedged_pct = usd_unhedged / total

# --- Alertas de drawdown ---
amber_alert = current_dd <= -AMBER_DD
red_alert = current_dd <= -MAX_DD

# --- Estado general de riesgo ---
if red_alert:
    estado_general = "ROJO"
elif amber_alert or len(breaches_asset) > 0 or len(breaches_region) > 0 or usd_unhedged_pct >= 0.10:
    estado_general = "AMBER"
else:
    estado_general = "VERDE"

# --- Dashboard ---
risk_dashboard = pd.DataFrame({
    "métrica": [
        "Drawdown actual",
        "Retorno anualizado cartera",
        "Retorno anualizado S&P 500",
        "Alpha vs S&P 500",
        "Alpha objetivo (+5%)",
        "Drawdown ≥ 9% (ámbar)",
        "Drawdown ≥ 10% (rojo)",
        "Activos > 5%",
        "Regiones > 30%",
        "USD no hedged (%)"
    ],
    "valor": [
        f"{current_dd:.1%}",
        f"{annual_return:.1%}",
        f"{sp500_return:.1%}" if pd.notna(sp500_return) else "N/A",
        f"{alpha:.1%}" if pd.notna(alpha) else "N/A",
        "✅" if pd.notna(alpha) and alpha >= 0.05 else "❌",
        amber_alert,
        red_alert,
        len(breaches_asset),
        len(breaches_region),
        f"{usd_unhedged_pct:.1%}"
    ],
    "estado": [
        "✅" if not amber_alert else "⚠️" if not red_alert else "🔴",
        "✅" if annual_return > 0 else "⚠️",
        "N/A",
        "✅" if pd.notna(alpha) and alpha > 0 else "⚠️",
        "✅" if pd.notna(alpha) and alpha >= 0.05 else "⚠️",
        "✅" if not amber_alert else "⚠️",
        "✅" if not red_alert else "🔴",
        "✅" if len(breaches_asset) == 0 else "⚠️",
        "✅" if len(breaches_region) == 0 else "⚠️",
        "✅" if usd_unhedged_pct < 0.10 else "⚠️"
    ]
})

print("=== 🛡️ DASHBOARD DE RIESGO (con Alpha vs S&P 500) ===")
print(risk_dashboard.to_string(index=False))

# --- Guardar ---
CSV_PATH = f"{DIRS['reports']}/risk_dashboard.csv"
risk_dashboard.to_csv(CSV_PATH, index=False)
print(f"\n✅ Dashboard guardado: {CSV_PATH}")

BREACHES_ASSET_PATH = f"{DIRS['reports']}/breaches_asset.csv"
breaches_asset.to_csv(BREACHES_ASSET_PATH, index=False)
print(f"✅ Brechas por activo: {BREACHES_ASSET_PATH}")

BREACHES_REGION_PATH = f"{DIRS['reports']}/breaches_region.csv"
breaches_region.to_csv(BREACHES_REGION_PATH, index=False)
print(f"✅ Brechas por región: {BREACHES_REGION_PATH}")

# --- Guardar resumen para orquestador ---
risk_summary = {
    "fecha": pd.Timestamp.now().strftime("%Y-%m-%d"),
    "estado_general": estado_general,
    "drawdown": float(current_dd),
    "retorno_cartera": float(annual_return),
    "alpha": float(alpha) if pd.notna(alpha) else None,
    "brechas": {
        "activos": len(breaches_asset),
        "regiones": len(breaches_region)
    },
    "usd_no_hedged_pct": float(usd_unhedged_pct)
}

JSON_PATH = f"{DIRS['reports']}/risk_dashboard_latest.json"
with open(JSON_PATH, "w") as f:
    json.dump(risk_summary, f, indent=2)
print(f"✅ Resumen para orquestador: {JSON_PATH}")

Mounted at /content/drive
=== 🛡️ DASHBOARD DE RIESGO (con Alpha vs S&P 500) ===
                   métrica valor estado
           Drawdown actual -3.8%      ✅
Retorno anualizado cartera  2.1%      ✅
Retorno anualizado S&P 500 11.5%    N/A
          Alpha vs S&P 500 -8.4%     ⚠️
      Alpha objetivo (+5%)     ❌     ⚠️
     Drawdown ≥ 9% (ámbar) False      ✅
     Drawdown ≥ 10% (rojo) False      ✅
              Activos > 5%     8     ⚠️
            Regiones > 30%     2     ⚠️
         USD no hedged (%) 26.0%     ⚠️

✅ Dashboard guardado: /content/drive/MyDrive/investment_ai/reports/risk_dashboard.csv
✅ Brechas por activo: /content/drive/MyDrive/investment_ai/reports/breaches_asset.csv
✅ Brechas por región: /content/drive/MyDrive/investment_ai/reports/breaches_region.csv
✅ Resumen para orquestador: /content/drive/MyDrive/investment_ai/reports/risk_dashboard_latest.json
